In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF
from nltk.tokenize import word_tokenize
import nltk
import operator

In [2]:
df_train = pd.read_csv('/Users/amitagarwal/GreyAtom/Hackathon/mckinesy_recommendation_dataset/train.csv')
df_test = pd.read_csv("/Users/amitagarwal/GreyAtom/Hackathon/mckinesy_recommendation_dataset/test.csv")

In [3]:
# Treat the sequence of challenges as text
df_train = df_train[df_train.challenge_sequence <= 13].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()

In [4]:
df_train.head()

,user_id,challenge
0,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...
1,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...
2,4581,CI26155 CI26156 CI26157 CI26158 CI26159 CI2616...
3,4582,CI23855 CI24915 CI24917 CI23933 CI23663 CI2495...
4,4585,CI23855 CI23975 CI24917 CI25135 CI23848 CI2371...


In [5]:
# Treat the sequence of challenges as text
df_test = df_test[df_test.challenge_sequence <= 10].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()

In [6]:
df_test.head()

,user_id,challenge
0,4577,CI23855 CI23933 CI24917 CI24915 CI23714 CI2366...
1,4578,CI23663 CI23855 CI24917 CI23933 CI23975 CI2371...
2,4579,CI26939 CI26940 CI26941 CI26942 CI26943 CI2694...
3,4583,CI23663 CI23855 CI23975 CI23714 CI23848 CI2393...
4,4584,CI23855 CI23975 CI25135 CI23848 CI23714 CI2491...


In [7]:
def matrix_creation(sentence):  
    dict_sen = dict()
    words = word_tokenize(sentence)
    for word in words:
        dict_sen[word] = 1
    return dict_sen

In [8]:
df_train['challenge'] = df_train['challenge'].apply(matrix_creation)

In [9]:
df_test['challenge'] = df_test['challenge'].apply(matrix_creation)

In [10]:
df_train.head()

,user_id,challenge
0,4576,"{'CI23714': 1, 'CI23855': 1, 'CI24917': 1, 'CI..."
1,4580,"{'CI23663': 1, 'CI23855': 1, 'CI23933': 1, 'CI..."
2,4581,"{'CI26155': 1, 'CI26156': 1, 'CI26157': 1, 'CI..."
3,4582,"{'CI23855': 1, 'CI24915': 1, 'CI24917': 1, 'CI..."
4,4585,"{'CI23855': 1, 'CI23975': 1, 'CI24917': 1, 'CI..."


In [11]:
df_test.head()

,user_id,challenge
0,4577,"{'CI23855': 1, 'CI23933': 1, 'CI24917': 1, 'CI..."
1,4578,"{'CI23663': 1, 'CI23855': 1, 'CI24917': 1, 'CI..."
2,4579,"{'CI26939': 1, 'CI26940': 1, 'CI26941': 1, 'CI..."
3,4583,"{'CI23663': 1, 'CI23855': 1, 'CI23975': 1, 'CI..."
4,4584,"{'CI23855': 1, 'CI23975': 1, 'CI25135': 1, 'CI..."


In [12]:
df_train = df_train.set_index('user_id', drop=True)

In [13]:
df_train.head()

,challenge
user_id,
4576,"{'CI23714': 1, 'CI23855': 1, 'CI24917': 1, 'CI..."
4580,"{'CI23663': 1, 'CI23855': 1, 'CI23933': 1, 'CI..."
4581,"{'CI26155': 1, 'CI26156': 1, 'CI26157': 1, 'CI..."
4582,"{'CI23855': 1, 'CI24915': 1, 'CI24917': 1, 'CI..."
4585,"{'CI23855': 1, 'CI23975': 1, 'CI24917': 1, 'CI..."


In [14]:
train_challenge_df = df_train['challenge'].apply(pd.Series)

In [15]:
train_challenge_df.head()

,CI23714,CI23855,CI24917,CI23663,CI23933,CI25135,CI23975,CI25126,CI24915,CI24957,...,CI27159,CI28672,CI26373,CI28667,CI28664,CI25931,CI28657,CI28611,CI25967,CI27413
user_id,,,,,,,,,,,,,,,,,,,,,
4576,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4580,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4582,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4585,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
train_challenge_df.shape

(69532, 5348)

In [17]:
df_test = df_test.set_index('user_id', drop=True)

In [18]:
test_challenge_df = df_test['challenge'].apply(pd.Series)

In [19]:
test_challenge_df.head()

,CI23855,CI23933,CI24917,CI24915,CI23714,CI23663,CI24958,CI25135,CI25727,CI24530,...,CI28235,CI28212,CI28213,CI28218,CI27593,CI27594,CI27114,CI23976,CI24820,CI24803
user_id,,,,,,,,,,,,,,,,,,,,,
4577,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4578,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4583,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4584,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df = pd.concat([train_challenge_df, test_challenge_df])

/Users/amitagarwal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [21]:
df.shape

(109264, 5502)

In [22]:
df = df.replace(np.nan, 0)

In [23]:
df.head()

,CI23478,CI23479,CI23480,CI23481,CI23482,CI23483,CI23484,CI23485,CI23486,CI23487,...,CI29073,CI29074,CI29075,CI29076,CI29078,CI29079,CI29080,CI29081,CI29082,CI29083
user_id,,,,,,,,,,,,,,,,,,,,,
4576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df = pd.concat([df[:25000], df[-100:]])

In [25]:
df.shape

(25100, 5502)

In [26]:
def get_nmf_matrix(V, k=10):
    nmf = NMF(k)
    nmf.fit(V)
    
    H = pd.DataFrame(np.round(nmf.components_,2), columns=V.columns)
    
    latent_features = list()
    for i in range(1, k+1):
        latent_features.append('latent_feature_' + str(i))
    
    H.index = latent_features
    
    W = pd.DataFrame(np.round(nmf.transform(V),2), columns=H.index)
    W.index = V.index
    
    reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=V.columns)
    reconstructed.index = V.index
    return reconstructed

In [27]:
result_df = get_nmf_matrix(df, 12)

In [28]:
result_df = result_df[-100:]

In [29]:
result_df = result_df.T

In [30]:
results_for_user_ids = result_df.to_dict()

In [31]:
filtered_results_for_user_id = dict()
for user_id in results_for_user_ids.keys():
    new_dict = dict()
    for challengeId in results_for_user_ids[user_id].keys():
        if(results_for_user_ids[user_id][challengeId] > 0):
            new_dict[challengeId] = results_for_user_ids[user_id][challengeId]
    
    filtered_results_for_user_id[user_id] = new_dict
    
filtered_results_for_user_id

{113559: {'CI23546': 0.06,
  'CI23598': 0.01,
  'CI23612': 0.02,
  'CI23648': 0.18,
  'CI23653': 0.03,
  'CI23662': 0.02,
  'CI23663': 0.09,
  'CI23667': 0.08,
  'CI23688': 0.02,
  'CI23691': 0.2,
  'CI23702': 0.05,
  'CI23714': 0.03,
  'CI23769': 0.1,
  'CI23770': 0.04,
  'CI23781': 0.04,
  'CI23790': 0.02,
  'CI23791': 0.01,
  'CI23830': 0.01,
  'CI23836': 0.03,
  'CI23839': 0.01,
  'CI23840': 0.01,
  'CI23848': 0.07,
  'CI23855': 0.03,
  'CI23856': 0.07,
  'CI23880': 0.07,
  'CI23887': 0.01,
  'CI23913': 0.06,
  'CI23922': 0.01,
  'CI23928': 0.01,
  'CI23932': 0.01,
  'CI23933': 0.15,
  'CI23975': 0.02,
  'CI23990': 0.01,
  'CI23999': 0.02,
  'CI24024': 0.01,
  'CI24029': 0.01,
  'CI24030': 0.02,
  'CI24031': 0.09,
  'CI24081': 0.01,
  'CI24111': 0.01,
  'CI24117': 0.06,
  'CI24120': 0.03,
  'CI24138': 0.12,
  'CI24139': 0.02,
  'CI24141': 0.01,
  'CI24175': 0.01,
  'CI24184': 0.1,
  'CI24187': 0.13,
  'CI24188': 0.02,
  'CI24206': 0.03,
  'CI24228': 0.12,
  'CI24231': 0.01,
  'CI24

In [32]:
new_filtered_results_for_user_id = dict()
for user_id in filtered_results_for_user_id.keys():
    new_dict = dict()
    challenges_solved_by_user = set(df_test.loc[[user_id]]['challenge'].values[0].keys())
    resultant_challenges_by_user = set(filtered_results_for_user_id[user_id].keys())
    
    challenges_not_solved = resultant_challenges_by_user - challenges_solved_by_user
    challenges_not_solved = list(challenges_not_solved)
    
    for challengeId in challenges_not_solved:
        new_dict[challengeId] = filtered_results_for_user_id[user_id][challengeId]
        
    

    sorted_new_dict = sorted(new_dict.items(), key=operator.itemgetter(1), reverse=True)
    new_filtered_results_for_user_id[user_id] = sorted_new_dict

In [33]:
new_filtered_results_for_user_id

{113559: [('CI23691', 0.2),
  ('CI24534', 0.2),
  ('CI24958', 0.2),
  ('CI25075', 0.19),
  ('CI23648', 0.18),
  ('CI24530', 0.15),
  ('CI23933', 0.15),
  ('CI24532', 0.14),
  ('CI24915', 0.13),
  ('CI24187', 0.13),
  ('CI24527', 0.13),
  ('CI24138', 0.12),
  ('CI24866', 0.12),
  ('CI24228', 0.12),
  ('CI24917', 0.12),
  ('CI24531', 0.11),
  ('CI24184', 0.1),
  ('CI23769', 0.1),
  ('CI24031', 0.09),
  ('CI23663', 0.09),
  ('CI24261', 0.08),
  ('CI23667', 0.08),
  ('CI25057', 0.08),
  ('CI23880', 0.07),
  ('CI23848', 0.07),
  ('CI23856', 0.07),
  ('CI25074', 0.06),
  ('CI23546', 0.06),
  ('CI23913', 0.06),
  ('CI24875', 0.06),
  ('CI24957', 0.05),
  ('CI25124', 0.05),
  ('CI23702', 0.05),
  ('CI23770', 0.04),
  ('CI25125', 0.04),
  ('CI24956', 0.04),
  ('CI26051', 0.04),
  ('CI25149', 0.04),
  ('CI24953', 0.04),
  ('CI23714', 0.03),
  ('CI24522', 0.03),
  ('CI24944', 0.03),
  ('CI24120', 0.03),
  ('CI24969', 0.03),
  ('CI25105', 0.03),
  ('CI24206', 0.03),
  ('CI24992', 0.03),
  ('CI2385

In [34]:
challenges_to_predict = dict()
for user_id in new_filtered_results_for_user_id.keys():
    challenges = list()
    for item in new_filtered_results_for_user_id[user_id][:3]:
        challenges.append(item[0])
    challenges_to_predict[user_id] = challenges

In [35]:
challenges_to_predict

{113559: ['CI23691', 'CI24534', 'CI24958'],
 113562: ['CI26219', 'CI26217', 'CI26218'],
 113565: ['CI23714', 'CI23855', 'CI23691'],
 113570: ['CI26910', 'CI26905', 'CI26907'],
 113572: ['CI24958', 'CI24917', 'CI24915'],
 113573: ['CI24534', 'CI24876', 'CI25075'],
 113577: [],
 113578: ['CI26203', 'CI26204', 'CI26202'],
 113579: ['CI24958', 'CI24917', 'CI24915'],
 113581: ['CI23855', 'CI24958', 'CI23714'],
 113582: ['CI24534', 'CI24876', 'CI23691'],
 113586: ['CI26931', 'CI26927', 'CI26929'],
 113589: ['CI26932', 'CI26922', 'CI26934'],
 113594: ['CI26931', 'CI26927', 'CI26929'],
 113595: ['CI26931', 'CI26929', 'CI26932'],
 113597: ['CI26931', 'CI26927', 'CI26926'],
 113599: ['CI26931', 'CI26929', 'CI26932'],
 113600: ['CI26931', 'CI26929', 'CI26928'],
 113603: ['CI26930', 'CI26927', 'CI26928'],
 113604: ['CI26930', 'CI26927', 'CI26929'],
 113606: ['CI26930', 'CI26929', 'CI26932'],
 113608: ['CI26926', 'CI26932', 'CI26934'],
 113613: ['CI26931', 'CI26927', 'CI26928'],
 113619: ['CI26907'

In [36]:
df_sub = pd.read_csv('/Users/amitagarwal/GreyAtom/Hackathon/mckinesy_recommendation_dataset/sub3.csv')


In [37]:
df_sub[['user_id', 'user_sequence']] = df_sub['user_sequence'].str.split('_', expand=True)

In [38]:
df_sub = df_sub[['user_id', 'challenge']]

In [39]:
df_sub

,user_id,challenge
0,4577,CI23848
1,4577,CI26051
2,4577,CI25124
3,4578,CI23648
4,4578,CI25141
5,4578,CI23667
6,4579,CI26950
7,4579,CI26958
8,4579,CI26945
9,4583,CI23691


In [40]:
# Treat the sequence of challenges as text
df_sub = df_sub.groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()

In [41]:
df_sub.head()

,user_id,challenge
0,100001,CI26922 CI26917 CI26915
1,100003,CI26931 CI26856 CI26924
2,100005,CI26906 CI26183 CI26909
3,100007,CI24436 CI24445 CI24373
4,100008,CI25124 CI26051 CI25727


In [42]:
def array_creation(sentence):  
    my_array = list()
    words = word_tokenize(sentence)
    for word in words:
        my_array.append(word)
    return my_array
df_sub['challenge'] = df_sub['challenge'].apply(array_creation)

In [43]:
df_sub

,user_id,challenge
0,100001,"[CI26922, CI26917, CI26915]"
1,100003,"[CI26931, CI26856, CI26924]"
2,100005,"[CI26906, CI26183, CI26909]"
3,100007,"[CI24436, CI24445, CI24373]"
4,100008,"[CI25124, CI26051, CI25727]"
5,100010,"[CI24120, CI23667, CI24527]"
6,100012,"[CI25295, CI24930, CI24690]"
7,100013,"[CI25126, CI26050, CI26056]"
8,100020,"[CI24690, CI24531, CI24958]"
9,100023,"[CI26929, CI26924, CI26208]"


In [44]:
df_sub['user_id'] = df_sub['user_id'].astype(int)

In [45]:


df_sub.dtypes

user_id       int64
challenge    object
dtype: object

In [46]:
df_sub = df_sub[df_sub['user_id'].isin(challenges_to_predict.keys())]

In [47]:
df_sub = df_sub.set_index('user_id', drop=True)

In [48]:
sub_dict = df_sub.T.to_dict()

In [49]:
challenges_actual = dict()
for user_id in sub_dict.keys():
    challenges_actual[user_id] = sub_dict[user_id]['challenge']

In [50]:
challenges_actual

{113559: ['CI24690', 'CI24930', 'CI24252'],
 113562: ['CI26164', 'CI26903', 'CI26161'],
 113565: ['CI24610', 'CI24440', 'CI24641'],
 113570: ['CI26919', 'CI26928', 'CI26916'],
 113572: ['CI27648', 'CI25634', 'CI27644'],
 113573: ['CI24032', 'CI24530', 'CI26912'],
 113577: ['CI26215', 'CI26220', 'CI26899'],
 113578: ['CI26158', 'CI26190', 'CI25634'],
 113579: ['CI25634', 'CI25119', 'CI24931'],
 113581: ['CI25148', 'CI25062', 'CI23688'],
 113582: ['CI25507', 'CI25443', 'CI25446'],
 113586: ['CI26871', 'CI26872', 'CI26869'],
 113589: ['CI26924', 'CI26937', 'CI26208'],
 113594: ['CI26869', 'CI26871', 'CI26864'],
 113595: ['CI26872', 'CI26871', 'CI26216'],
 113597: ['CI26931', 'CI26927', 'CI26924'],
 113599: ['CI26871', 'CI26872', 'CI26931'],
 113600: ['CI26870', 'CI26871', 'CI26864'],
 113603: ['CI26928', 'CI27644', 'CI26911'],
 113604: ['CI26911', 'CI26924', 'CI26167'],
 113606: ['CI26924', 'CI26930', 'CI26936'],
 113608: ['CI26924', 'CI26937', 'CI26926'],
 113613: ['CI26928', 'CI26931', 

In [51]:
"""Ben Hammer metrics page
"""
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [52]:
precision = list()
for user_id in challenges_actual.keys():
    precision.append(apk(challenges_actual[user_id], challenges_to_predict[user_id]))

In [53]:
np.mean(precision)

0.08666666666666666